In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
dataset = pd.read_csv('bookdata.csv') 
dataf=dataset.drop(['Unnamed: 0'], axis=1)

dataf.head()

,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


In [7]:
df= dataf.copy()
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df.drop(columns = ['location','isbn','book_author',
                   'img_m','age','Language',
                   'year_of_publication'],axis=1,inplace = True) #remove useless cols

df.drop(index=df[df['rating'] == 0].index, inplace=True) #remove 0 in rating
df.head()

,user_id,rating,book_title,publisher,img_s,img_l,Summary,Category,city,state,country
1,8,5,Clara Callan,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",['Actresses'],timmins,ontario,canada
4,67544,8,Clara Callan,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",['Actresses'],toronto,ontario,canada
7,123629,9,Clara Callan,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",['Actresses'],kingston,ontario,canada
9,200273,8,Clara Callan,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",['Actresses'],comber,ontario,canada
10,210926,9,Clara Callan,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",['Actresses'],guelph,ontario,canada


In [16]:
num_rating= df.groupby('book_title').count()['rating'].reset_index()
num_rating.rename(columns={'rating': 'num_ratings'}, inplace=True)

avg_rating= df.groupby('book_title').mean()['rating'].reset_index()
avg_rating.rename(columns={'rating': 'avg_ratings'}, inplace=True)

C:\Users\TISHA\AppData\Local\Temp\ipykernel_27816\4152258272.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating= df.groupby('book_title').mean()['rating'].reset_index()


# Popular Books

In [17]:
popular= num_rating.merge(avg_rating, on='book_title')
popular[popular['num_ratings']>=1].sort_values('avg_ratings')
popular_df= popular[popular['num_ratings']>=180].sort_values('avg_ratings', ascending= False).head(50)
popular_df.merge(dataset, on='book_title').drop_duplicates('book_title').shape
popular_df= popular_df.merge(dataset, on='book_title').drop_duplicates('book_title')[['book_title','book_author','img_m','num_ratings', 'avg_ratings']]

In [18]:
popular_df

,book_title,book_author,img_m,num_ratings,avg_ratings
0,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,233,9.154506
387,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,263,9.076046
815,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,202,9.039604
1162,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,254,8.996063
1672,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,298,8.919463
2247,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,http://images.amazon.com/images/P/0618002227.0...,194,8.896907
2615,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064864.0...,313,8.843450
3171,Fahrenheit 451,Ray Bradbury,http://images.amazon.com/images/P/8401422825.0...,195,8.569231
3580,"Tuesdays with Morrie: An Old Man, a Young Man,...",MITCH ALBOM,http://images.amazon.com/images/P/0385484518.0...,236,8.529661
4073,The Secret Life of Bees,Sue Monk Kidd,http://images.amazon.com/images/P/0142001740.0...,391,8.480818


In [11]:
x= df.groupby('user_id').count()['rating']>150
experienced_users= x[x].index

filtered_on_users= df[df['user_id'].isin(experienced_users)]

y= filtered_on_users.groupby('book_title').count()['rating']>=8
famous_books= y[y].index # boolean indexing

final_ratings= filtered_on_users[filtered_on_users['book_title'].isin(famous_books)]
final_ratings.drop_duplicates
final_ratings

,user_id,rating,book_title,publisher,img_s,img_l,Summary,Category,city,state,country
31,46398,9,The Kitchen God's Wife,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],san antonio,texas,usa
50,230522,7,The Kitchen God's Wife,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,A Chinese immigrant who is convinced she is dy...,['Fiction'],colorado springs,colorado,usa
97,7346,9,The Testament,Dell,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,"A suicidal billionaire, a burnt-out Washington...",['Fiction'],sunnyvale,california,usa
105,13552,8,The Testament,Dell,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,"A suicidal billionaire, a burnt-out Washington...",['Fiction'],cordova,tennessee,usa
130,31315,10,The Testament,Dell,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,"A suicidal billionaire, a burnt-out Washington...",['Fiction'],henryville,indiana,usa
...,...,...,...,...,...,...,...,...,...,...,...
961758,241980,10,Desperation,Sagebrush Bound,http://images.amazon.com/images/P/0613025024.0...,http://images.amazon.com/images/P/0613025024.0...,9,9,ashburn,virginia,usa
961760,241980,9,Bag of Bones,Sagebrush Bound,http://images.amazon.com/images/P/0613171004.0...,http://images.amazon.com/images/P/0613171004.0...,9,9,ashburn,virginia,usa
962046,258534,9,Insomnia,Dutton Books,http://images.amazon.com/images/P/0451186125.0...,http://images.amazon.com/images/P/0451186125.0...,"Or, at least, he hopes they are hallucinations...",['English fiction'],mcqueeney,texas,usa
966483,248718,8,Ashes to Ashes,Horizon Book Promotions,http://images.amazon.com/images/P/031202164X.0...,http://images.amazon.com/images/P/031202164X.0...,9,9,hyde park,new york,usa


In [12]:
pt= final_ratings.pivot_table(index= 'book_title', columns= 'user_id', values='rating')
pt.fillna(0, inplace= True)
pt

user_id,2276,4017,4385,6251,6543,6575,7346,12982,13552,16634,...,264082,264321,265115,265889,268932,269566,270713,273113,274061,274301
book_title,,,,,,,,,,,,,,,,,,,,,
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong to Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0


# Recommend Books

In [13]:
similarity_score= cosine_similarity(pt)
similarity_score.shape #book-to-book

def recommend(book_title):
    # index number fetch
    index= np.where(pt.index== book_title)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key= lambda x:x[1], reverse= True)[1:6]
    data=[]
    for i in similar_items:
        item=[]
        #print(pt.index[i[0]])
        temp_df =dataset[dataset['book_title']== pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('book_title')['book_title'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['book_author'].values))
        item.extend(list(temp_df.drop_duplicates('book_title')['img_m'].values))

        data.append(item)
    return data

In [14]:
recommend("The Notebook")

[['The Rescue',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446610399.01.MZZZZZZZ.jpg'],
 ['A Walk to Remember',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446525537.01.MZZZZZZZ.jpg'],
 ['Message in a Bottle',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446523569.01.MZZZZZZZ.jpg'],
 ["Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series (Paper))",
  'Jack Canfield',
  'http://images.amazon.com/images/P/1558744150.01.MZZZZZZZ.jpg'],
 ['Carnal Innocence',
  'Nora Roberts',
  'http://images.amazon.com/images/P/0553295977.01.MZZZZZZZ.jpg']]

In [15]:
recommend("Interview with the Vampire")

[['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
  'Anne Rice',
  'http://images.amazon.com/images/P/034538475X.01.MZZZZZZZ.jpg'],
 ['The Scarlet Letter',
  'NATHANIEL HAWTHORNE',
  'http://images.amazon.com/images/P/0553210092.01.MZZZZZZZ.jpg'],
 ['Wizard and Glass (The Dark Tower, Book 4)',
  'Stephen King',
  'http://images.amazon.com/images/P/0451194861.01.MZZZZZZZ.jpg'],
 ['1984',
  'George Orwell',
  'http://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg']]

# Genre-Based Books

In [26]:
def recomendCategory(df, category):
    filtered_df = df[df['Category'] == category]
    filtered_df = filtered_df[filtered_df['rating'] >=9]
    filtered_df = filtered_df.drop_duplicates(subset='book_title')
    filtered_df = filtered_df.merge(dataf, on='book_title').drop_duplicates('book_title')[['book_title', 'book_author', 'img_m']]
    return filtered_df


medical_df = recomendCategory(df, "['Medical']")
fiction_df = recomendCategory(df, "['Fiction']")
history_df = recomendCategory(df, "['History']")
selfhelp_df = recomendCategory(df, "['Self-Help']")
cooking_df = recomendCategory(df, "['Cooking']")

In [27]:
medical_df

,book_title,book_author,img_m
0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,http://images.amazon.com/images/P/0374157065.0...
11,The Man Who Mistook His Wife for a Hat: And Ot...,Oliver Sacks,http://images.amazon.com/images/P/0060970790.0...
51,The Seven Sins of Memory: How the Mind Forgets...,Daniel L. Schacter,http://images.amazon.com/images/P/0618219196.0...
55,The Womanly Art of Breastfeeding,Gwen Gotsch,http://images.amazon.com/images/P/0452279089.0...
64,The Vaccine Guide: Making an Informed Choice,Randall Neustaedter,http://images.amazon.com/images/P/1556432151.0...
...,...,...,...
607,Scourge: The Once and Future Threat of Smallpox,Jonathan B. Tucker,http://images.amazon.com/images/P/0871138301.0...
608,Why Survive?: Being Old in America,"Robert N., MD Butler",http://images.amazon.com/images/P/0801874254.0...
609,The New Diary of a Teenage Health Freak,Aidan MacFarlane,http://images.amazon.com/images/P/0192861824.0...
610,Breastfeeding Special Care Babies,Sandra Lang,http://images.amazon.com/images/P/0702020206.0...


In [28]:
cooking_df

,book_title,book_author,img_m
0,New Vegetarian: Bold and Beautiful Recipes for...,Celia Brooks Brown,http://images.amazon.com/images/P/1841721522.0...
37,From the Earth: Chinese Vegetarian Cooking,Eileen Yin-Fei Lo,http://images.amazon.com/images/P/0026329859.0...
38,Totally Muffins Cookbook (Totally Cookbooks),Helene Siegel,http://images.amazon.com/images/P/0890877564.0...
40,Fg on Our Immigrant Ance,J Smith,http://images.amazon.com/images/P/0380717085.0...
44,The Northern Exposure Cookbook: A Community Co...,Ellis Weiner,http://images.amazon.com/images/P/0809237601.0...
...,...,...,...
1284,Martini (Expressions),Alexander B. Struminger,http://images.amazon.com/images/P/1577170229.0...
1285,Eli's Wings,Elizabeth Best,http://images.amazon.com/images/P/0141004606.0...
1286,"Mr. Food-Fun Kitchen Tips (And Recipes, Too!)",Art Ginsburg,http://images.amazon.com/images/P/0688137105.0...
1287,"Build Your Own Earth Oven : A Low-Cost, Wood-F...",Kiko Denzer,http://images.amazon.com/images/P/0967984602.0...


In [21]:
fiction_df

,book_title,book_author,img_m
0,The Kitchen God's Wife,Amy Tan,http://images.amazon.com/images/P/0399135782.0...
311,The Testament,John Grisham,http://images.amazon.com/images/P/0440234743.0...
928,Beloved (Plume Contemporary Fiction),Toni Morrison,http://images.amazon.com/images/P/0452264464.0...
1108,Wild Animus,Rich Shapero,http://images.amazon.com/images/P/0971880107.0...
3610,Airframe,Michael Crichton,http://images.amazon.com/images/P/0345402871.0...
...,...,...,...
333495,Smoke and Gravity,Win Neagle,http://images.amazon.com/images/P/0970172605.0...
333497,Murder in the Mews and Other Stories (Hercule ...,Agatha Christie,http://images.amazon.com/images/P/0785748814.0...
333498,The WEIGHT OF WINTER,Cathie Pelletier,http://images.amazon.com/images/P/067179387X.0...
333499,The Runes of Elfland,Ari Berk,http://images.amazon.com/images/P/0810946122.0...


In [29]:
history_df

,book_title,book_author,img_m
0,Seabiscuit: An American Legend,LAURA HILLENBRAND,http://images.amazon.com/images/P/0449005615.0...
275,Seabiscuit,LAURA HILLENBRAND,http://images.amazon.com/images/P/0345465083.0...
462,Midnight in the Garden of Good and Evil: A Sav...,John Berendt,http://images.amazon.com/images/P/0679429220.0...
727,The Courage of Their Convictions: Sixteen Amer...,Peter H. Irons,http://images.amazon.com/images/P/0140128107.0...
729,Hidden in Plain View: A Secret Story of Quilts...,Jacqueline L. Tobin,http://images.amazon.com/images/P/0385497679.0...
...,...,...,...
4522,Night of Stone: Death and Memory in Twentieth-...,Catherine Merridale,http://images.amazon.com/images/P/0670894745.0...
4523,The Wit & Wisdom of Winston Churchill,James C. Humes,http://images.amazon.com/images/P/0060925779.0...
4524,The Trial of Socrates,I.F. Stone,http://images.amazon.com/images/P/0316817589.0...
4527,My Heart Soars,Dan George,http://images.amazon.com/images/P/0888392311.0...


In [30]:
selfhelp_df

,book_title,book_author,img_m
0,Chicken Soup for the Soul (Chicken Soup for th...,Jack Canfield,http://images.amazon.com/images/P/155874262X.0...
205,The Four Agreements: A Practical Guide to Pers...,Don Miguel Ruiz,http://images.amazon.com/images/P/1878424319.0...
338,Chicken Soup for the College Soul : Inspiring ...,Jack Canfield,http://images.amazon.com/images/P/1558747028.0...
390,Writing Down the Bones,NATALIE GOLDBERG,http://images.amazon.com/images/P/0877733759.0...
440,"14,000 Things to Be Happy About",Barbara Ann Kipfer,http://images.amazon.com/images/P/0894803700.0...
...,...,...,...
3127,The Excuse Me Your Life is Waiting Playbook,Lynn Grabhorn,http://images.amazon.com/images/P/1571742700.0...
3128,Self-Esteem Comes in All Sizes,Carol Johnson,http://images.amazon.com/images/P/0385475691.0...
3129,Heal Your Heartbreak: How to Live and Love Again,Chuck Spezzano,http://images.amazon.com/images/P/1569246262.0...
3130,Is It Too Late to Run Away and Join the Circus...,Marti Diane Smye,http://images.amazon.com/images/P/0028620585.0...
